In [19]:
import os, json, uuid, random
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
from pymongo import MongoClient
from bson import json_util

load_dotenv()
uri = os.getenv("MONGODB_URI")
database = os.getenv("MONGODB_DATABASE")
client = MongoClient(uri, tz_aware=True)
coll = client[database]["pump_inventories"]

def _num(x):
    try:
        return float(str(x))
    except:
        return None

def _dt(x):
    if isinstance(x, datetime):
        return x
    try:
        return datetime.fromisoformat(str(x).replace("Z","+00:00"))
    except:
        return None

def _new_8_digit_code():
    while True:
        code = str(random.randint(10_000_000, 99_999_999))
        if not coll.find_one({"current_form_qr_code": code}):
            return code

def _already_has_next(cur, expected_next_date=None):
    q = {
        "previous_form_qr_code": cur.get("current_form_qr_code",""),
        "location": cur.get("location"),
        "dispenser_name": cur.get("dispenser_name"),
        "pump_id": cur.get("pump_id"),
    }
    if expected_next_date is not None:
        q["date"] = expected_next_date
    return coll.find_one(q) is not None

def create_followup_forms(current_form_qr_code: str, check_date=True, update_source=True):
    now = datetime.now(timezone.utc)
    src_docs = list(coll.find({"current_form_qr_code": current_form_qr_code}))
    created, skipped = [], []
    for cur in src_docs:
        base_date = _dt(cur.get("date")) or now
        next_date = base_date + timedelta(days=1)
        if _already_has_next(cur, next_date if check_date else None):
            skipped.append({
                "source_id": cur.get("id"),
                "reason": "next_form_already_exists",
                "previous_form_qr_code": cur.get("current_form_qr_code")
            })
            continue
        new_doc = {
            "id": str(uuid.uuid4()),
            "created_at": now,
            "location": cur.get("location"),
            "date": next_date,
            "dispenser_name": cur.get("dispenser_name"),
            "pump_id": cur.get("pump_id"),
            "pump_name": cur.get("pump_name"),
            "product": cur.get("product"),
            "unit": cur.get("unit"),
            "price": _num(cur.get("price")),
            "beginning_inventory": _num(cur.get("ending_inventory")),
            "starting_liter_meter": _num(cur.get("ending_liter_meter")),
            "current_form_qr_code": _new_8_digit_code(),
            "previous_form_qr_code": cur.get("current_form_qr_code","")
        }
        coll.insert_one(new_doc)
        if update_source:
            coll.update_one(
                {"_id": cur["_id"]},
                {"$set": {"next_form_qr_code": new_doc["current_form_qr_code"]}}
            )
        created.append(new_doc)
    return {
        "source_current_form_qr_code": current_form_qr_code,
        "created_count": len(created),
        "skipped_count": len(skipped),
        "created": created,
        "skipped": skipped
    }

result = create_followup_forms("11458998", check_date=True, update_source=True)
print(json.dumps(result, default=json_util.default, ensure_ascii=False, indent=2))


{
  "source_current_form_qr_code": "11458998",
  "created_count": 0,
  "skipped_count": 4,
  "created": [],
  "skipped": [
    {
      "source_id": "9a755455-2557-4a51-8980-a7a995c55bdb",
      "reason": "next_form_already_exists",
      "previous_form_qr_code": "11458998"
    },
    {
      "source_id": "ab0f39cf-9bf8-452b-b806-aea262cc0a16",
      "reason": "next_form_already_exists",
      "previous_form_qr_code": "11458998"
    },
    {
      "source_id": "5881e035-7735-40b0-a796-6baf2849c756",
      "reason": "next_form_already_exists",
      "previous_form_qr_code": "11458998"
    },
    {
      "source_id": "5e40ebe1-8707-4cc0-b644-b4e92258b4f8",
      "reason": "next_form_already_exists",
      "previous_form_qr_code": "11458998"
    }
  ]
}
